In [10]:
import pandas as pd

In [11]:
from tqdm.notebook import tqdm
import sys
import os
import re
from glob import glob
from time import sleep

prefix = os.getcwd().split("jdi-qasp-ml")[0]
sys.path.append(os.path.join(prefix, "jdi-qasp-ml"))

In [12]:
dataset_path = os.path.join(prefix, "jdi-qasp-ml", "data/mui_dataset")

SITE_DFS = [
    p
    for p in glob(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/mui_dataset/df/site*')
]
DF_NAMES = [re.search("site-[0-9]+", nm)[0] for nm in SITE_DFS]

In [13]:
# total_df = pd.DataFrame()
# for i, df_path in tqdm(enumerate(SITE_DFS)):
#     df = pd.read_pickle(df_path)
#     df['site'] = DF_NAMES[i]

#     par_childs = {par:list(df[df.parent_id==par].element_id.unique()) for par in df.parent_id.unique()}

#     def find_all_childs(elem_id):
#         if par_childs.get(elem_id) is None:
#             return []
#         else:
#             res = []
#             for i in par_childs.get(elem_id):
#                 res.append(i)
#                 res.extend(find_all_childs(i))
#             return res
    
#     df['all_followers'] = df['element_id'].apply(find_all_childs)

#     df['STRING'] = df.attributes.apply((lambda x: '' if x is None else x.get('class', "") + " " + x.get('type', "") + " " + x.get('role', "")))
#     df['STRING'] = df.tag_name.str.lower().str.cat(df['STRING'], sep=" ")
#     df['LABEL'] = df.attributes.apply(lambda x: None if x is None else x.get('data-label'))
#     df = df[["site", "element_id", "parent_id", "all_followers", "STRING", "LABEL"]]
#     df.reset_index(drop=True, inplace=True)

#     elem_string_dict = {k:v.get('STRING') for k,v in df[['STRING']].set_index(df.element_id).fillna('').T.to_dict().items()}

#     def concat_all_strings(list):
#         res = ''
#         for l in list:
#             res = res + elem_string_dict.get(l) + ' '
#         return res
    
#     df['all_followers_string'] = df['all_followers'].apply(concat_all_strings)
#     df['final_string'] = (df.STRING + ' ' + df.all_followers_string).fillna('')

#     df = df[["site", "element_id", "parent_id", "final_string", "LABEL"]]
#     total_df = pd.concat([total_df, df])
# total_df.reset_index(inplace=True, drop=True)

In [14]:
# total_df.tail(3)

In [15]:
# total_df.to_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/total_df_for_nlp_model.csv', index=False)

In [16]:
# total_df = pd.read_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/total_df_for_nlp_model.csv')
# total_df.LABEL = total_df.LABEL.fillna("n/a")


In [17]:
# total_df.tail()

In [18]:
def prepare_string(string):
    string = re.sub(r"(?<=\w)([A-Z])", r" \1", string)
    string = str(string).lower().strip()
    return(re.sub(' +', ' ', re.sub('(-)|([0-9]+)', ' ', string).strip()))


# total_df['final_string'] = total_df['final_string'].apply(prepare_string)

In [19]:
# total_df.tail()

In [20]:
# total_df.to_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/total_df_for_nlp_model_prepared.csv', index=False)

In [21]:
total_df = pd.read_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/total_df_for_nlp_model_prepared.csv')
total_df.LABEL = total_df.LABEL.fillna("n/a")

In [22]:
total_df.tail()

,site,element_id,parent_id,final_string,LABEL
739896,site-13,9627524316559287764950874349,8293726446559287768833831671,div,n/a
739897,site-13,9638773789559287760038756767,8293726446559287768833831671,div,n/a
739898,site-13,4889127663559287761125737880,8293726446559287768833831671,div mui paper root mui popover paper mui paper...,n/a
739899,site-13,8651010676559287765452773916,4889127663559287761125737880,p mui typography root jss mui typography body,typography
739900,site-13,9980562521559287761155767932,8293726446559287768833831671,div,n/a


In [23]:
train_dfs = DF_NAMES[:400]
val_dfs = DF_NAMES[400:450]
test_dfs = DF_NAMES[450:]

train_df = total_df[total_df.site.isin(train_dfs)].copy()[['element_id', 'final_string', 'LABEL']].copy().reset_index(drop=True)
val_df = total_df[total_df.site.isin(val_dfs)].copy()[['element_id', 'final_string', 'LABEL']].copy().reset_index(drop=True)
test_df = total_df[total_df.site.isin(test_dfs)].copy()[['element_id', 'final_string', 'LABEL']].copy().reset_index(drop=True)

# train.to_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/'+ 'nlp_train.csv', index=False)
# val_df.to_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/'+ 'nlp_val.csv', index=False)
# test_df.to_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/'+ 'nlp_test.csv', index=False)

In [24]:
train_df

,element_id,final_string,LABEL
0,5377782770559833469033671578,html head meta link meta meta meta link link t...,n/a
1,7829459489559833462475342827,head meta link meta meta meta link link title ...,n/a
2,0631196382559833465296481442,meta,n/a
3,3432217540559833464644554437,link,n/a
4,8074386043559833462890934880,meta,n/a
...,...,...,...
588254,7082420646580084644836977116,div,n/a
588255,4245731393580084649975250848,div,n/a
588256,8235246246580084649721106092,div mui paper root mui popover paper jss mui p...,n/a
588257,1458830904580084640254145688,p mui typography root mui typography body,n/a


In [25]:
train_df[train_df.element_id=="6533486216559833460616205753"]

,element_id,final_string,LABEL
112,6533486216559833460616205753,span mui slider root mui slider color primary ...,slider


Create text pipeline

In [26]:
import itertools

vocab = list(set(list(
            itertools.chain.from_iterable(
                train_df.final_string.apply(lambda x: x.split(sep=" "))
            )
        )))

In [27]:
print(vocab)

['viewport', 'scroller', 'entered', 'contained', 'breadcrumbs', 'data', 'contract', 'select', 'tr', 'warning', 'badge', 'fixed', 'switch', 'tooltip', 'pickers', 'day', 'alternative', 'em', 'renderer', 'error', 'animate', 'padding', 'xs', 'wrapper', 'txt', 'direction', 'spacing', 'resize', 'span', 'fieldset', 'view', 'deletable', 'horizontal', 'paper', 'actions', 'disabled', 'completed', 'chip', 'subtitle', 'clock', 'edge', 'link', 'footer', 'active', 'rectangle', 'delete', 'helper', 'text/css', 'top', 'time', 'backdrop', 'base', 'html', 'position', 'center', 'pin', 'anchor', 'email', 'multiline', 'summary', 'expanded', 'required', 'placement', 'success', 'mask', 'separator', 'details', 'justify', 'primary', 'triggers', 'only', 'left', 'btn', 'mui', 'gutter', 'svg', 'password', 'legend', 'asterisk', 'body', 'color', 'inset', 'max', 'mark', 'app', 'h', 'checkbox', 'radiogroup', 'desktop', 'pagination', 'columnheader', 'window', 'collapse', 'header', 'days', 'value', 'scroll', 'numeric', 

In [28]:
print(len(vocab))


277


In [29]:
classes_path = os.path.join(prefix, "jdi-qasp-ml", "data/mui_dataset/classes.txt")
with open(classes_path, "r") as f:
    lines = f.readlines()
    classes_dict = {v.strip(): i for i, v in enumerate(lines)}
    classes_reverse_dict = {i: v.strip() for i, v in enumerate(lines)}

# train_df.LABEL = train_df.LABEL.apply(lambda x: classes_dict.get(x, 0))

In [30]:
classes_dict

{'n/a': 0,
 'link': 1,
 'bottom-navigation': 2,
 'menu': 3,
 'stepper': 4,
 'progress': 5,
 'dialog': 6,
 'button': 7,
 'switch': 8,
 'table': 9,
 'breadcrumbs': 10,
 'list': 11,
 'alert': 12,
 'chip': 13,
 'select': 14,
 'textarea-autosize': 15,
 'accordion': 16,
 'text-field': 17,
 'radio': 18,
 'checkbox': 19,
 'drawer': 20,
 'snackbar': 21,
 'radiogroup': 22,
 'slider': 23,
 'badge': 24,
 'button-group': 25,
 'tabs': 26}

In [31]:
text_pipeline = lambda x: prepare_string(x)
label_pipeline = lambda x: classes_dict.get(x, 0)

In [32]:
text_pipeline("div MuiPaper-root MuiPopover-paper MuiPaper")

'div mui paper root mui popover paper mui paper'

Generate data batch and iterator

In [33]:
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfTransformer,
)

In [34]:

model_cv = CountVectorizer(vocabulary=vocab).fit(train_df.final_string.values)
cv_transformed = model_cv.transform(train_df.final_string.values)
model_tf = TfidfTransformer().fit(cv_transformed)
tfidf_transformed = model_tf.transform(cv_transformed)


In [35]:
X_train = tfidf_transformed.copy()
y_train = train_df.LABEL.apply(lambda x: classes_dict.get(x, 0)).astype(int)

cv_transformed = model_cv.transform(val_df.final_string.values)
X_val = model_tf.transform(cv_transformed)
y_val = val_df.LABEL.apply(lambda x: classes_dict.get(x, 0)).astype(int)

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
pd.Series(y_train).value_counts()

0     543393
7      10993
13      7273
18      5599
19      3536
1       2614
11      2104
16      1876
5       1097
24       820
8        715
17       652
15       592
12       579
21       576
4        575
26       556
2        545
10       543
20       538
23       532
3        525
22       513
9        499
14       498
25       393
6        123
Name: LABEL, dtype: int64

In [38]:
# classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
# classifier.fit(X_train, y_train) 
# import pickle
# with open("RF_model.pkl", "wb") as c:
#             pickle.dump(classifier, c)
#             c.flush()

In [40]:
import pickle
with open("RF_model.pkl", "rb") as f:
            classifier = pickle.load(f)

In [41]:
y_pred = classifier.predict(X_val)

In [42]:
val_df.loc[123]

element_id                           3256084674564724863972029713
final_string    span mui button base root mui icon button root...
LABEL                                                       radio
Name: 123, dtype: object

In [43]:
pr = pd.Series(y_pred).apply(lambda x: classes_reverse_dict.get(x))
pr[pr!='n/a']

Series([], dtype: object)

In [95]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [44]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(classification_report(y_val,y_pred))
print(accuracy_score(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     70201
           1       0.00      0.00      0.00       296
           2       0.00      0.00      0.00        91
           3       0.00      0.00      0.00        70
           4       0.00      0.00      0.00        57
           5       0.00      0.00      0.00       134
           6       0.00      0.00      0.00        14
           7       0.00      0.00      0.00      1412
           8       0.00      0.00      0.00       104
           9       0.00      0.00      0.00        45
          10       0.00      0.00      0.00        66
          11       0.00      0.00      0.00       280
          12       0.00      0.00      0.00        46
          13       0.00      0.00      0.00      1084
          14       0.00      0.00      0.00        70
          15       0.00      0.00      0.00        71
          16       0.00      0.00      0.00       218
          17       0.00    

/Users/Mikhail_Bulgakov/opt/anaconda3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Mikhail_Bulgakov/opt/anaconda3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Mikhail_Bulgakov/opt/anaconda3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [45]:
prefix = os.getcwd().split('jdi-qasp-ml')[0]
sys.path.append(os.path.join(prefix, 'jdi-qasp-ml'))
from utils.dataset_builder import DatasetBuilder
from utils.common import maximize_window 

2021-11-30 14:07:51,428 -INFO - config.py:31 - Module utils.config was loaded
2021-11-30 14:07:51,547 -INFO - common.py:119 - Module utils.common is loaded...
2021-11-30 14:07:51,548 -INFO - dataset_builder.py:245 - dataset package is loaded...


In [46]:
DATASET_NAME = 'material-ui-Buttons'
WAIT_TIME_SECONDS = 7
SITE_URL = 'https://jdi-testing.github.io/jdi-light/'
# SITE_URL = 'https://jdi-testing.github.io/jdi-light/material/buttons'
PREDICT_URL = 'http://localhost:5050/mui-predict'

LOGIN = 'Roman'
PASSWORD = 'Jdi1234'
SAVE_SCREEN = True
WAIT_TIME_SECONDS = 7

In [75]:
class DSBuilder(DatasetBuilder):
    def setUp(self, driver):
        """
            This method may be overwitten if you need to login to the site
            and navigate to a specific url,
            otherwise it just returns 
        """
        
        driver.set_window_size(width=1500, height=15000)
        self.driver.get(self.url)
        driver.find_element_by_id("user-icon").click()
        driver.find_element_by_id("name").send_keys(LOGIN)
        driver.find_element_by_id("password").send_keys(PASSWORD)
        driver.find_element_by_id("login-button").click()
        sleep(WAIT_TIME_SECONDS)
        driver.find_element_by_xpath("/html/body/div/div[1]/div/div[1]/div/div[1]/ul/li[5]/a/div").click()
        driver.find_element_by_xpath("/html/body/div/div[1]/div/div[1]/div/div[1]/ul/li[5]/ul/li[8]/a").click()
        driver.find_element_by_xpath("/html/body/div/div/div/div[1]/div/div[2]/div/div").click()
        sleep(3.0)
        maximize_window(driver=driver, extend_pix=0)
        sleep(7.0)
        
with DSBuilder(url=SITE_URL, dataset_name=DATASET_NAME, headless=True, dataset_root_path=dataset_path) as builder:
    df = builder.dataset

2021-11-30 14:18:59,467 -INFO - dataset_builder.py:100 - Dataset name: material-ui-Buttons
2021-11-30 14:18:59,470 -INFO - dataset_builder.py:110 - Create directories to save the dataset
2021-11-30 14:18:59,472 -INFO - dataset_builder.py:151 - Creating driver
2021-11-30 14:19:01,315 -INFO - dataset_builder.py:163 - Chrome web driver is created
2021-11-30 14:19:14,183 -INFO - common.py:22 - Window maximized
2021-11-30 14:19:21,190 -INFO - dataset_builder.py:166 - save scrinshot: /Users/Mikhail_Bulgakov/GitRepo/jdi-qasp-ml/data/mui_dataset/images/material-ui-Buttons.png
2021-11-30 14:19:27,810 -INFO - dataset_builder.py:210 - Collect features using JS script
2021-11-30 14:19:28,442 -INFO - dataset_builder.py:218 - Save html to /Users/Mikhail_Bulgakov/GitRepo/jdi-qasp-ml/data/mui_dataset/html/material-ui-Buttons.html
2021-11-30 14:19:28,460 -INFO - dataset_builder.py:230 - Save peakle to /Users/Mikhail_Bulgakov/GitRepo/jdi-qasp-ml/data/mui_dataset/df/material-ui-Buttons.pkl
2021-11-30 14:

In [67]:
df = builder.dataset

In [76]:
df["site"] = "mui_buttons"
par_childs = {par:list(df[df.parent_id==par].element_id.unique()) for par in df.parent_id.unique()}

def find_all_childs(elem_id):
    if par_childs.get(elem_id) is None:
        return []
    else:
        res = []
        for i in par_childs.get(elem_id):
            res.append(i)
            res.extend(find_all_childs(i))
        return res

df['all_followers'] = df['element_id'].apply(find_all_childs)

df['STRING'] = df.attributes.apply((lambda x: '' if x is None else x.get('class', "") + " " + x.get('type', "") + " " + x.get('role', "")))
df['STRING'] = df.tag_name.str.lower().str.cat(df['STRING'], sep=" ")
df['LABEL'] = df.attributes.apply(lambda x: None if x is None else x.get('data-label'))
df = df[["site", "element_id", "parent_id", "all_followers", "STRING", "LABEL"]]
df.reset_index(drop=True, inplace=True)

elem_string_dict = {k:v.get('STRING') for k,v in df[['STRING']].set_index(df.element_id).fillna('').T.to_dict().items()}

def concat_all_strings(list):
    res = ''
    for l in list:
        res = res + elem_string_dict.get(l) + ' '
    return res

df['all_followers_string'] = df['all_followers'].apply(concat_all_strings)
df['final_string'] = (df.STRING + ' ' + df.all_followers_string).fillna('').apply(prepare_string)

df = df[["site", "element_id", "parent_id", "final_string", "LABEL"]]

/var/folders/n6/xlg_j8tx7xzdv6qgxc7hwb680000gp/T/ipykernel_15378/4177210050.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['all_followers_string'] = df['all_followers'].apply(concat_all_strings)
/var/folders/n6/xlg_j8tx7xzdv6qgxc7hwb680000gp/T/ipykernel_15378/4177210050.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['final_string'] = (df.STRING + ' ' + df.all_followers_string).fillna('').apply(prepare_string)


In [77]:
df[df.final_string.apply(lambda x: 'button' in x)]

,site,element_id,parent_id,final_string,LABEL
0,mui_buttons,0269594694711678175970615340,None,html head link meta title meta meta noscript l...,None
120,mui_buttons,9015416235711678175666305286,0269594694711678175970615340,body div div div mui grid root mui grid contai...,None
121,mui_buttons,1486404568711678175511849197,9015416235711678175666305286,div div div mui grid root mui grid container d...,None
122,mui_buttons,7965836976711678176671339181,1486404568711678175511849197,div div mui grid root mui grid container div m...,None
123,mui_buttons,0350515143711678170641231175,7965836976711678176671339181,div mui grid root mui grid container div mui g...,None
...,...,...,...,...,...
609,mui_buttons,0819828816711678184474173763,7822840155711678186134499331,span mui button label,None
612,mui_buttons,7062622000711678185935999164,2592352868711678185001627240,div jss button mui button base root jss button...,None
613,mui_buttons,4235694777711678186599470765,7062622000711678185935999164,button mui button base root jss button span js...,None
620,mui_buttons,2487698244711678184322260342,7062622000711678185935999164,button mui button base root jss button span js...,None


In [78]:
cv_transformed = model_cv.transform(df.final_string.values)
X_buttons = model_tf.transform(cv_transformed)

In [87]:
X_buttons.todense()[627]

matrix([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.43308756, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.25468926, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.10809318, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0

In [82]:
X_buttons.todense()[-1]

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [91]:
buttons_pred = classifier.predict_proba(X_buttons)

In [93]:
buttons_pred

array([[0.67889694, 0.        , 0.        , ..., 0.        , 0.        ,
        0.059     ],
       [0.97605546, 0.        , 0.015     , ..., 0.        , 0.        ,
        0.        ],
       [0.97605546, 0.        , 0.015     , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])